In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/kaggle/input/symptoms-and-covid-presence/Covid Dataset.csv')

In [ ]:
df

### Data Cleaning 

In [ ]:
df.info()

In [ ]:
df = df.replace({'No': 0, 'Yes': 1})

In [ ]:
df.columns = df.columns.str.replace(' ', '')

In [ ]:
df.info()

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
features = df[['BreathingProblem','Fever','DryCough','Sorethroat','RunningNose','Asthma','ChronicLungDisease','Headache','HeartDisease','Diabetes','HyperTension','Fatigue','Abroadtravel','ContactwithCOVIDPatient','WearingMasks']]
label = df['COVID-19']

In [ ]:
train_x_df,test_x_df, train_y_df, test_y_df = train_test_split(features,label,test_size=0.30,random_state=10)
train_df = pd.concat([train_x_df,train_y_df],axis=1)
test_df = pd.concat([test_x_df,test_y_df],axis=1)

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
import tensorflow_decision_forests as tfdf

# **Training**

In [ ]:
train_tf = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="COVID-19")
test_tf = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="COVID-19")

In [ ]:
tfdf.keras.get_all_models() 

## RandomForestModel Tensorflow

In [ ]:
model = tfdf.keras.RandomForestModel(num_trees=150)
model.fit(train_tf)
model.compile(metrics=["accuracy"])
model.evaluate(test_tf)

In [ ]:
import matplotlib.pyplot as plt
model.make_inspector().evaluation()

logs = model.make_inspector().training_logs()

plt.figure(figsize=(24, 8))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

        
# plt.subplot(1, 2, 2)
# plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
# plt.xlabel("Number of trees")
# plt.ylabel("Logloss (out-of-bag)")

plt.show()

### GradientBoostedTreesModel

In [ ]:

model_1 = tfdf.keras.GradientBoostedTreesModel(num_trees=100)
model_1.fit(train_tf)

model_1.compile(metrics=["accuracy"])
acc = model_1.evaluate(test_tf)
print('Accuracy: '+str(acc[1]*100))

In [ ]:
import matplotlib.pyplot as plt
model_1.make_inspector().evaluation()

logs = model_1.make_inspector().training_logs()

plt.figure(figsize=(24, 8))

plt.subplot(1, 2, 1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

# plt.subplot(1, 2, 2)
# plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
# plt.xlabel("Number of trees")
# plt.ylabel("Logloss (out-of-bag)")

plt.show()

### CartModel - Classification

In [ ]:
model_3 = tfdf.keras.CartModel()
model_3.fit(train_tf)
model_3.compile(metrics=["accuracy"])
acc = model_3.evaluate(test_tf)
print('Accuracy: '+str(acc[1]*100))

## Decision Tree Classifier

In [ ]:
train_x,test_x, train_y, test_y = train_test_split(features,label,test_size=0.30,random_state=10)


In [ ]:
from sklearn import tree
model_4 = tree.DecisionTreeClassifier()
model_4 = model_4.fit(train_x,train_y)
y_pred = model_4.predict(test_x)

from sklearn.metrics import accuracy_score

accuracy_score(test_y, y_pred)
print('Accuracy: '+str(accuracy_score(test_y, y_pred)*100))


In [ ]:
test_x.head(1)

In [ ]:
np.array([1,1,1,1,0,1,1,1,1,1,1,1,1,1]).reshape(1,-1)

In [ ]:
#15

model_4.predict(np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]).reshape(1,-1))

## Saving the model

In [ ]:
import pickle

In [ ]:
pickleModel = 'model.sav'
pickle.dump(model_4, open(pickleModel,'wb'))
#model is saved in our directory as model.sav we can use it in our flask api now